In [7]:
import ray
import time
import numpy as np
from copy import deepcopy # 모델 파라미터를 복사하기 위한 라이브러리 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [2]:
ray.init()

2021-01-29 12:50:57,890	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.22',
 'raylet_ip_address': '192.168.0.22',
 'redis_address': '192.168.0.22:6379',
 'object_store_address': 'tcp://127.0.0.1:52941',
 'raylet_socket_name': 'tcp://127.0.0.1:58691',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\KUKJIN~1\\AppData\\Local\\Temp\\ray\\session_2021-01-29_12-50-55_923578_39060',
 'metrics_export_port': 43394,
 'node_id': '53bfe28a944112d73498a91621f9bd1db6fd29e6'}

2021-01-29 12:51:01,840	WARNING worker.py:1091 -- The dashboard on node DESKTOP-ESV82HP failed with the following error:
Traceback (most recent call last):
  File "c:\users\kukjinkim\anaconda3\envs\minerl\lib\site-packages\ray\dashboard\dashboard.py", line 962, in <module>
    metrics_export_address=metrics_export_address)
  File "c:\users\kukjinkim\anaconda3\envs\minerl\lib\site-packages\ray\dashboard\dashboard.py", line 512, in __init__
    build_dir = setup_static_dir(self.app)
  File "c:\users\kukjinkim\anaconda3\envs\minerl\lib\site-packages\ray\dashboard\dashboard.py", line 411, in setup_static_dir
    "&& npm run build)", build_dir)
FileNotFoundError: [Errno 2] Dashboard build directory not found. If installing from source, please follow the additional steps required to build the dashboard(cd python/ray/dashboard/client && npm ci && npm run build): 'c:\\users\\kukjinkim\\anaconda3\\envs\\minerl\\lib\\site-packages\\ray\\dashboard\\client/build'



In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class A3C(nn.Module):
    def __init__(self, num_actions):
        super(A3C, self).__init__()

        self.num_actions = num_actions

        self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=4)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(64)

        def conv2d_size_out(size, kernel_size=3, stride=2):
            return (size - (kernel_size - 1) - 1) // stride + 1

        convw = conv2d_size_out(210, 8, 4)
        convw = conv2d_size_out(convw, 4, 2)
        convw = conv2d_size_out(convw, 3, 1)

        convh = conv2d_size_out(160, 8, 4)
        convh = conv2d_size_out(convh, 4, 2)
        convh = conv2d_size_out(convh, 3, 1)

        linear_input_size = convw * convh * 64
        print("linear_input_size", linear_input_size)
        self.lstm_i_dim = 512  # input dimension of LSTM
        self.lstm_h_dim = 512  # output dimension of LSTM
        self.lstm_N_layer = 1  # number of layers of LSTM
        self.Conv2LSTM = nn.Linear(linear_input_size, self.lstm_i_dim)
        self.lstm = nn.LSTM(input_size=self.lstm_i_dim, hidden_size=self.lstm_h_dim, num_layers=self.lstm_N_layer)

        self.fc_pi = nn.Linear(self.lstm_h_dim, self.num_actions)
        self.fc_v = nn.Linear(self.lstm_h_dim, 1)

    def pi(self, x, softmax_dim=1):
        x = self.fc_pi(x)
        prob = F.softmax(x, dim=softmax_dim)
        return prob

    def v(self, x):
        v = self.fc_v(x)
        return v

    def forward(self, x, hidden, softmax_dim=2):
        x = x/255.0
        if (len(x.shape) < 4):  # 배치학습이 아닐 때
            x = x.unsqueeze(0)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = x.contiguous()  # x torch.Size([1, 64, 22, 16])
        x = x.view(x.size(0), -1)  # x torch Size([1, 22528])
        x = F.relu(self.Conv2LSTM(x))
        x = x.unsqueeze(1)  # x torch Size([1,1,1024])
        x, new_hidden = self.lstm(x, hidden)
        return x, new_hidden


In [14]:
brain = A3C(num_actions=4).to(device)
input_data = torch.randn(size=(3, 210, 160)).to(device)
hidden = (Variable(torch.zeros(1, 1, 512).float().to(device=device)),Variable(torch.zeros(1, 1, 512).float().to(device=device)))
print(input_data)
print(input_data.shape)
out_data, new_hidden = brain(data, hidden)
print(out_data)
print(out_data.shape)

linear_input_size 22528
tensor([[[-0.0097, -0.7700, -0.2625,  ..., -1.8247,  0.7748,  0.2273],
         [-0.5518, -1.1822,  0.0720,  ...,  1.3301, -0.0274,  0.5967],
         [-0.9266,  0.1560,  0.4429,  ...,  0.9406, -0.4538,  1.5117],
         ...,
         [-0.9680,  1.7315, -1.0140,  ..., -0.6431, -0.4889, -0.1594],
         [ 0.0366, -0.2669, -1.5177,  ..., -2.2529, -1.1450, -1.4923],
         [ 0.9742,  0.4543, -0.3285,  ..., -0.6975, -0.0850,  2.0572]],

        [[ 1.1047, -2.2310, -0.2101,  ...,  1.0271, -0.8119, -0.6909],
         [ 0.5769,  1.5306, -0.0727,  ..., -0.0725,  0.2916, -0.6817],
         [-0.0883, -0.7771, -0.7282,  ...,  0.4471,  0.0279, -0.0940],
         ...,
         [ 0.8975, -2.6726, -1.1230,  ..., -0.1625, -1.5088, -0.0213],
         [ 0.2520, -0.7394, -0.1420,  ...,  0.3636,  0.5944,  0.1619],
         [-0.5319, -0.2252, -1.1208,  ...,  1.6820,  0.9883, -0.4077]],

        [[-0.4958,  0.2344, -0.0498,  ...,  1.2414,  0.1514, -1.1733],
         [ 1.3670,  1

In [20]:
@ray.remote
class Parameter_server:
    def __init__(self):
        self.params = []
    # 푸쉬와 풀, 업데이트의 개념을 잘 생각
    
    def updates_params(self, model):
        self.params.append(model)
    
    def pull_params(self):
        return self.params[-1]

In [22]:
@ray.remote
class Actor:
    def __init__(self, params_server, actor_idx):
        self.params_server = params_server
        self.actor_idx = actor_idx # 몇 번째 에이전트인지 인덱스
        self.device = 'cpu' # Actor의 프로세스는 cpu에 할당한다. Learner의 경우 gpu로 학습
        self.actor_network = A3C(num_actions=4).to(self.device)
        self.counter = 0
    def explore(self): 
        while 1: # 카운터 변수를 통해 탐색 횟수 정하기?
            time.sleep(5)
            print("Explore..")
            self.pull_params()
        pass
    
    def get_params_server(self):
        return self.params_server
    
    def pull_params(self):
        updated_params = ray.get(self.params_server.pull_params.remote()) #업데이트된 파라미터를 파라미터 서버로부터 가져온다
        print("Parameter Type : ", type(updated_params))
        self.actor_network.load_state_dict(updated_params)
        print(f"Model i supdated in actor number_{self.actor_idx}")
        

In [25]:
class Learner:
    def __init__(self, params_server):
        self.params_server = params_server
        self.device = 'cuda:0'
        
        self.learner_network = A3C(num_actions=4).to(self.device)
    
    def get_params_server(self):
        return self.params_server
    
    def push_parameters(self):
        self.params_server.updates_params.remote(self.learner_network.cpu().state_dict())
        
    def pull_parameters_from_server(self):
        return self.params_server.pull_params.remote()
    
    def train(self):
        while 1:
            time.sleep(2)
            self.push_parameters()
            print("Learner: Model parameter is sent to Server")

In [30]:
# 파라미터 서버 선언
params_server = Parameter_server.remote()

learner = Learner(params_server)
learner2 = Learner(params_server)
actor = Actor.remote(params_server, 0)

print(learner.get_params_server())
print(learner2.get_params_server())
#print(ray.get(actor.get_params_server.remote()))

linear_input_size 22528
linear_input_size 22528
Actor(Parameter_server,ae935fc001000000)
Actor(Parameter_server,ae935fc001000000)
(pid=4276) linear_input_size 22528


In [31]:
# model 업데이트를 3회 진행
iteration = 3
for _ in range(iteration): learner.push_parameters()

In [ ]:
# learner에서 server의 파라미터를 가지고 와보는지 test, 실제로 러너는 서버에서 가지고 올 일이 없다.
ray.get(learner.pull_parameters_from_server())

In [35]:
# actor의 explore 메소드를 통해 learner로 부터 받은 parameter를 server를 통해 가져오고, actor의 모델로 저장이 되는지 테스트
actor.explore.remote()

ObjectRef(84b65a94b96102243db7cfef0100000001000000)

(pid=4276) Explore..
(pid=4276) Parameter Type :  <class 'collections.OrderedDict'>
(pid=4276) Model i supdated in actor number_0
(pid=4276) Explore..
(pid=4276) Parameter Type :  <class 'collections.OrderedDict'>
(pid=4276) Model i supdated in actor number_0
(pid=4276) Explore..
(pid=4276) Parameter Type :  <class 'collections.OrderedDict'>
(pid=4276) Model i supdated in actor number_0
(pid=4276) Explore..
(pid=4276) Parameter Type :  <class 'collections.OrderedDict'>
(pid=4276) Model i supdated in actor number_0
(pid=4276) Explore..
(pid=4276) Parameter Type :  <class 'collections.OrderedDict'>
(pid=4276) Model i supdated in actor number_0
(pid=4276) Explore..
(pid=4276) Parameter Type :  <class 'collections.OrderedDict'>
(pid=4276) Model i supdated in actor number_0
(pid=4276) Explore..
(pid=4276) Parameter Type :  <class 'collections.OrderedDict'>
(pid=4276) Model i supdated in actor number_0
(pid=4276) Explore..
(pid=4276) Parameter Type :  <class 'collections.OrderedDict'>
(pid=4

In [36]:
ray.shutdown()

(pid=4276) Explore..


In [37]:
ray.init()

2021-01-29 13:50:10,702	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.22',
 'raylet_ip_address': '192.168.0.22',
 'redis_address': '192.168.0.22:6379',
 'object_store_address': 'tcp://127.0.0.1:34771',
 'raylet_socket_name': 'tcp://127.0.0.1:22897',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\KUKJIN~1\\AppData\\Local\\Temp\\ray\\session_2021-01-29_13-50-09_806950_39060',
 'metrics_export_port': 18979,
 'node_id': '3dfb39f49a0423c8102bab082bf47c05157616d7'}

2021-01-29 13:50:14,119	WARNING worker.py:1091 -- The dashboard on node DESKTOP-ESV82HP failed with the following error:
Traceback (most recent call last):
  File "c:\users\kukjinkim\anaconda3\envs\minerl\lib\site-packages\ray\dashboard\dashboard.py", line 962, in <module>
    metrics_export_address=metrics_export_address)
  File "c:\users\kukjinkim\anaconda3\envs\minerl\lib\site-packages\ray\dashboard\dashboard.py", line 512, in __init__
    build_dir = setup_static_dir(self.app)
  File "c:\users\kukjinkim\anaconda3\envs\minerl\lib\site-packages\ray\dashboard\dashboard.py", line 411, in setup_static_dir
    "&& npm run build)", build_dir)
FileNotFoundError: [Errno 2] Dashboard build directory not found. If installing from source, please follow the additional steps required to build the dashboard(cd python/ray/dashboard/client && npm ci && npm run build): 'c:\\users\\kukjinkim\\anaconda3\\envs\\minerl\\lib\\site-packages\\ray\\dashboard\\client/build'

(pid=None) F0129 14:48:46.111907 